In [2]:
import numpy as np
from scipy.sparse.linalg import eigs
import pickle
import os


path = os.path.expanduser("MBPR15/Benutzer/willi/Dokumente/PycharmProjects/Masterarbeit/RC/reservoir_preparations")


# simulation parameters
T = 625.0 # time
dt = 1e-3 # timesteps
dts = 1e-2 # timesteps, at which data gets stored
cutoff = 125.0  # cutoff time
n_epochs = 20
n_time = T/(n_epochs)
m = 6 # number of input channels (?)
m_n_time = T/(m*n_epochs)
m_n_epochs = (m*n_epochs)

# network configuration parameters
N = 800  # size of network
p = 0.2 # sparsity


n_alphas = 10
n_etas = 1
etas = np.linspace(-0.3, -0.3, num=n_etas)
alphas = [0.3]
etas_all = np.tile(etas, reps=(n_alphas,)) #tile(A,reps) constructs an array by repeating A the number of times given by reps.

# input parameters
a = 10.0
b = 28.0
c = 8/3
omega = 2*np.pi/5

# spike train input parameter
spike_probability = 1

# setup connectivity matrix
neurons = np.arange(0, N) # Return evenly spaced values within a given interval with stepsize 1 as an array
C = np.random.uniform(low=1e-4, high=1, size=(N, N)) # draws samples form uniform distribution from low to high as an ndarray, with size size
n_incoming = int(N*(1-p)) # int() returns an integer object out of a number or a string
for i in range(N):
    C[np.random.choice(neurons, size=n_incoming, replace=False), i] = 0 # randomly sets values in connectivity matrix to zero, according to sparsity p
vals, vecs = eigs(C, k=int(N/10)) # finds k eigenvalues 'vals' and k eigenvectors 'vecs' of C
sr = np.max(np.real(vals)) # finds highest of k eigenvalues
C /= sr # sets the spectral radius of C to 1

# setup input matrix
p_in = 0.1 # sparsity of input matrix
W_in = np.random.randn(N, m) # creates random matrix out of normal distribution
W_sorted = np.sort(W_in.flatten()) # np.sort sorts an array according to size, while .flatten collapses the array into 1 dimension
idx = np.abs(W_in) < W_sorted[int(N*m*p_in)] # abs() transforms complex elements in an array into their absolute value and gives out an array
#so this function captures all elements that are smaller than the part p with the biggest elements
W_in[idx] = 0.0 # and sets them zero
idx2 = np.abs(W_in) >= W_sorted[int(N*m*p_in)]
W_in[idx2] = np.random.uniform(-1, 1, np.sum(idx2)) # and the rest is drawn from a uniform distribution from -1 to 1.

#create poisson spike train sequences
train_length = 1000 # timesteps comprised in train
time_step = dt
timespan = train_length * time_step


#####input & target array####
m_n_steps = int(np.round(m_n_time/dt)) # timesteps of one input train epoche
n_steps = int(np.round(n_time/dt))
m_n_store_steps = int(np.round(m_n_time/dts))
train_reservoir = np.zeros((m, m_n_steps))
single_spike_train = np.zeros((train_length))

# create poisson input and target matrix
timesteps = int(np.round(T/dt)) # number of timesteps total
store_steps = int(np.round((T-cutoff)/dts))
in_starts = np.arange(0, timesteps, 2*m_n_steps) # 1darray from 0 to total time steps in steps of 2* in_dur, so number of input iterations
sequence_number = int(np.floor_divide(m_n_steps, train_length))

inp = np.full((m, timesteps), 0)

spike_train_target = np.zeros((0, store_steps))
targets = np.full((m, store_steps), -1)

for y in range(m):

    random_train_sequence = np.random.uniform(0, 1, (train_length))

    for i in range(0, train_length):
        if random_train_sequence[i] > spike_probability:
                single_spike_train[i] = 0

        if random_train_sequence[i] <= spike_probability:
                single_spike_train[i] = 1

    for r in range(0, sequence_number):
        for j in range(0, train_length):
            train_reservoir[y,j+r*train_length] = single_spike_train[j]

    i = y
    for N in range(0, n_epochs):
        for M in range(0, m):
            for j in range(0, m_n_steps-1):
                #inp[y, j+m*r*in_dur-1] = train_reservoir[y, j]
                #inp[y, i*m*r*m_n_steps-1:(i+1)*m*r*m_n_steps-1] = train_reservoir[y, :]
                inp[y, j+(N*n_steps)+(y*m_n_steps-1)] = train_reservoir[y, j]
                #inp[1, j+(N*n_steps)+(y*m_n_steps-1)] = train_reservoir

                if j+N*m*m_n_steps * dt >= cutoff:
                    targets[y, i * m_n_store_steps:(i + 1) * m_n_store_steps] = 1.0
                    i += m

inp = inp.astype(float)

# store data
data = {}
data['T'] = T
data['dt'] = dt
data['dts'] = dts
data['cutoff'] = cutoff
data['N'] = N
data['p'] = p
data['C'] = C
data['W_in'] = W_in
data['inp'] = inp
data['targets'] = targets
data['etas'] = etas_all
data['alphas'] = alphas
data['number_input_channels'] = m
pickle.dump(data,open('qif_input_config.pkl', 'wb'))

